##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Treine e sirva um modelo do TensorFlow com o TensorFlow Serving

**Importante: este notebook foi projetado para ser executado apenas num Google Colab**. Ele instala pacotes no sistema e requer acesso root. Se você quiser executá-lo num notebook Jupyter local, proceda com cuidado.

Observação: você pode executar este exemplo agora mesmo num notebook estilo Jupyter, sem necessidade de configuração! Basta clicar em "Executar no Google Colab"

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<tr>
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/serving/rest_simple"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/serving/rest_simple.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
<td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/serving/rest_simple.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Ver fonte no GitHub</a>
</td>
<td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tfx/tutorials/serving/rest_simple.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</tr>
</table></div>

Este guia treina um modelo de rede neural para classificar [imagens de roupas, como tênis e camisas](https://github.com/zalandoresearch/fashion-mnist), salva o modelo treinado e depois o disponibiliza com o [TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving). O foco está no TensorFlow Serving, em vez da modelagem e treinamento no TensorFlow, portanto, para obter um exemplo completo dedicado à modelagem e treinamento, veja o [Exemplo Básico de Classificação](https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/r1/tutorials/keras/basic_classification.ipynb).

Este guia usa o [tf.keras](https://github.com/tensorflow/docs/blob/master/site/en/r1/guide/keras.ipynb), uma API de alto nível, para criar e treinar modelos no TensorFlow.

In [ ]:
import sys

# Confirm that we're using Python 3
assert sys.version_info.major == 3, 'Oops, not running Python 3. Use Runtime > Change runtime type'

In [ ]:
# TensorFlow and tf.keras
print("Installing dependencies for Colab environment")
!pip install -Uq grpcio==1.26.0

import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

print('TensorFlow version: {}'.format(tf.__version__))

## Crie seu modelo

### Importe o dataset Fashion MNIST

Este guia usa o dataset [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) que contém 70.000 imagens em tons de cinza em 10 categorias. As imagens mostram peças de roupa individuais em baixa resolução (28 por 28 pixels), como pode ser visto aqui:

<table>
  <tr><td>     <img src="https://tensorflow.org/images/fashion-mnist-sprite.png" width="600" alt="Fashion MNIST sprite">
</td></tr>
  <tr><td align="center">     <b>Figura 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Amostras Fashion-MNIST</a> (por Zalando, licença MIT).<br>
</td></tr>
</table>

O Fashion MNIST pretende ser um substituto imediato para o dataset clássico [MNIST](http://yann.lecun.com/exdb/mnist/) - frequentemente usado como o "Olá, mundo" em programas de aprendizado de máquina para visão computacional. Você pode acessar o Fashion MNIST diretamente do TensorFlow, basta importar e carregar os dados.

Observação: Embora sejam realmente imagens, elas são carregadas como matrizes NumPy e não como objetos de imagem binária.

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# scale the values to 0.0 to 1.0
train_images = train_images / 255.0
test_images = test_images / 255.0

# reshape for feeding into the model
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print('\ntrain_images.shape: {}, of {}'.format(train_images.shape, train_images.dtype))
print('test_images.shape: {}, of {}'.format(test_images.shape, test_images.dtype))

### Treine e avalie seu modelo

Vamos usar a CNN mais simples possível, já que não estamos focados na parte de modelagem.

In [ ]:
model = keras.Sequential([
  keras.layers.Conv2D(input_shape=(28,28,1), filters=8, kernel_size=3, 
                      strides=2, activation='relu', name='Conv1'),
  keras.layers.Flatten(),
  keras.layers.Dense(10, name='Dense')
])
model.summary()

testing = False
epochs = 5

model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.SparseCategoricalAccuracy()])
model.fit(train_images, train_labels, epochs=epochs)

test_loss, test_acc = model.evaluate(test_images, test_labels)
print('\nTest accuracy: {}'.format(test_acc))

## Salve seu modelo

Para carregar nosso modelo treinado no TensorFlow Serving, primeiro precisamos salvá-lo no formato [SavedModel](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/saved_model). Isto criará um arquivo protobuf numa hierarquia de diretórios bem definida e incluirá um número de versão. [O TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving) nos permite selecionar qual versão de um modelo, ou "servable" ("servível"), queremos usar quando fazemos solicitações de inferência. Cada versão será exportada para um subdiretório diferente no caminho fornecido.

In [ ]:
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
import tempfile

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

## Examine seu modelo salvo

Usaremos o utilitário de linha de comando `saved_model_cli` para examinar os [MetaGraphDefs](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/MetaGraphDef) (os modelos) e [SignatureDefs](https://www.tensorflow.org/tfx/serving/signature_defs) (os métodos que você pode chamar) em nosso SavedModel. Veja [esta discussão sobre a CLI SavedModel](https://github.com/tensorflow/docs/blob/master/site/en/r1/guide/saved_model.md#cli-to-inspect-and-execute-savedmodel) no Guia do TensorFlow.

In [ ]:
!saved_model_cli show --dir {export_path} --all

Isto nos diz bastante sobre o nosso modelo! Neste caso acabamos de treinar nosso modelo, então já conhecemos as entradas e saídas, mas se não o fizéssemos, esta seria uma informação importante. Isto não nos diz tudo, como o fato de que se trata de dados de imagens em tons de cinza, por exemplo, mas é um ótimo começo.

## Sirva seu modelo com o TensorFlow Serving

**Importante: Se você NÃO estiver executando isso num Google Colab,** as células a seguir instalarão pacotes no sistema com acesso root. Se você quiser executá-lo num notebook Jupyter local, proceda com cuidado.

### Adicione a URI de distribuição do TensorFlow Serving como origem do pacote:

Estamos nos preparando para instalar o TensorFlow Serving usando o [Aptitude](https://wiki.debian.org/Aptitude), já que este Colab é executado em um ambiente Debian. Adicionaremos o pacote `tensorflow-model-server` à lista de pacotes que o Aptitude conhece. Observe que estamos executando como root.

Observação: este exemplo executa o TensorFlow Serving de forma nativa, mas [você também pode executá-lo num container Docker](https://www.tensorflow.org/tfx/serving/docker), que é uma das maneiras mais fáceis de começar a usar o TensorFlow Serving.

In [ ]:
import sys
# We need sudo prefix if not on a Google Colab.
if 'google.colab' not in sys.modules:
  SUDO_IF_NEEDED = 'sudo'
else:
  SUDO_IF_NEEDED = ''

In [ ]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | {SUDO_IF_NEEDED} tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | {SUDO_IF_NEEDED} apt-key add -
!{SUDO_IF_NEEDED} apt update

### Instale o TensorFlow Serving

Isso é tudo que você precisa: uma única linha de comando!

In [ ]:
# TODO: Use the latest model server version when colab supports it.
#!{SUDO_IF_NEEDED} apt-get install tensorflow-model-server
# We need to install Tensorflow Model server 2.8 instead of latest version
# Tensorflow Serving >2.9.0 required `GLIBC_2.29` and `GLIBCXX_3.4.26`. Currently colab environment doesn't support latest version of`GLIBC`,so workaround is to use specific version of Tensorflow Serving `2.8.0` to mitigate issue.
!wget 'http://storage.googleapis.com/tensorflow-serving-apt/pool/tensorflow-model-server-2.8.0/t/tensorflow-model-server/tensorflow-model-server_2.8.0_all.deb'
!dpkg -i tensorflow-model-server_2.8.0_all.deb
!pip3 install tensorflow-serving-api==2.8.0

### Comece a executar o TensorFlow Serving

É aqui que começamos a executar o TensorFlow Serving e carregamos nosso modelo. Após carregar, podemos começar a fazer solicitações de inferência usando REST. Existem alguns parâmetros importantes:

- `rest_api_port`: a porta que você usará para solicitações REST.
- `model_name`: você usará isto na URL das solicitações REST. Pode ser qualquer coisa.
- `model_base_path`: Este é o caminho para o diretório onde você salvou seu modelo.


In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=fashion_model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1


In [ ]:
!tail server.log

## Faça uma solicitação ao seu modelo no TensorFlow Serving

Primeiro, vamos dar uma olhada num exemplo aleatório de nossos dados de teste.

In [ ]:
def show(idx, title):
  plt.figure()
  plt.imshow(test_images[idx].reshape(28,28))
  plt.axis('off')
  plt.title('\n\n{}'.format(title), fontdict={'size': 16})

import random
rando = random.randint(0,len(test_images)-1)
show(rando, 'An Example Image: {}'.format(class_names[test_labels[rando]]))

Ok, isto parece interessante. Você acha difícil reconhecer isso? Agora vamos criar o objeto JSON para um lote de três solicitações de inferência e ver como nosso modelo reconhece as coisas:

In [ ]:
import json
data = json.dumps({"signature_name": "serving_default", "instances": test_images[0:3].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))

### Faça solicitações REST

#### Versão mais recente do servable

Enviaremos uma solicitação de previsão como uma solicitação POST para o endpoint REST do nosso servidor e passaremos três exemplos. Pediremos ao nosso servidor que nos forneça a versão mais recente do nosso servable, não especificando uma versão específica.

In [ ]:
# docs_infra: no_execute
!pip install -q requests

import requests
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/fashion_model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

show(0, 'The model thought this was a {} (class {}), and it was actually a {} (class {})'.format(
  class_names[np.argmax(predictions[0])], np.argmax(predictions[0]), class_names[test_labels[0]], test_labels[0]))

#### Uma versão específica do servable

Agora vamos especificar uma versão específica do nosso servable. Como só temos um, vamos selecionar a versão 1. Também veremos todos os três resultados.

In [ ]:
# docs_infra: no_execute
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/fashion_model/versions/1:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

for i in range(0,3):
  show(i, 'The model thought this was a {} (class {}), and it was actually a {} (class {})'.format(
    class_names[np.argmax(predictions[i])], np.argmax(predictions[i]), class_names[test_labels[i]], test_labels[i]))